In [ ]:
import os
#os.chdir('') #your directory
print(os.getcwd())

In [ ]:
#EDA
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Модели
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.linear_model import LogisticRegression

#Всопомгательные пакеты
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, validation_curve
from sklearn.metrics import roc_auc_score, plot_confusion_matrix, \
plot_roc_curve, plot_precision_recall_curve, classification_report, precision_score, recall_score


#Кластеризация
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors


In [ ]:
df = pd.read_csv('../input/bank-customer-churn-modeling/Churn_Modelling.csv')


Базовые описательные статистики

In [ ]:
df.describe()

In [ ]:
df.rename(columns = {'Exited':'target'}, inplace = True)

Пропусков нет, это хорошо =)

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

Количество уникальных значений для каждой колонки

In [ ]:
df.nunique()

Выбосим неинформативные, на первый взгляд признаки изи датафрейма

In [ ]:
df = df.drop(['RowNumber', 'CustomerId'], axis = 1)

Посмотрим на процентное соотношение классов целевой переменной

In [ ]:
ax = sns.countplot(y="target", data=df, alpha=0.8)
total = df.shape[0]

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width() / total)
    x = p.get_x() + p.get_width()
    y = p.get_y() + p.get_height() / 2
    ax.annotate(percentage, (x, y))

plt.show()

Дисбаланс классов вообще понятие эмперическое, в нашем случае можно в ходе настройки алгоритмов попробовать техники over и under sampling, а так же настройки с весами классов в самих алгоритмах

Сделаем мапинг нашей переменной Gender, с помощью двоичной кодировки

In [ ]:
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})

Сделаем OHE преобразование для переменной Geography, дамми-ловушка нас не пугает ибо будем пользоватся нелиейным алгоритмом (в итоге, скорее всего), поэтому не удаляем никакую из колонок

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Geography'])], axis = 1)
df.drop(columns = ['Geography', 'Surname'], inplace = True)

In [ ]:
df #посмотреть на выбросы Balance и CreditScore и EstimatedSalary

In [ ]:
#с выбросами не успел повозиться, можно по z score или по IQR их детектить, как вариант
#sns.boxplot(x="variable", y="value", data=pd.melt(df[["Balance", "CreditScore"]]))

In [ ]:
# plt.rcParams["figure.figsize"] = (15,10)
# fig, ax = plt.subplots(2)
# ax[0].boxplot(df['CreditScore'])
# ax[1].boxplot(df['Balance'])
# ax[0].grid()
# ax[1].grid()
# fig.show()

Рассмотрим гипотезу о том что те, у кого на балансе 0 и при этом есть кредитные карты более склонны к дефолту

In [ ]:
print(f"Число клиентов с нулевым балансом и кредитками - {len(df.query('Balance == 0 & HasCrCard != 0'))}")
display(df.query("Balance == 0 & HasCrCard != 0").head())
print()
print(f"Число клиентов с ненулевым балансом и отсутствием кредиткок - {len(df.query('Balance != 0 & HasCrCard == 0'))}")
display(df.query("Balance != 0 & HasCrCard == 0").head())

Как мы видим у нас примерно поровну тех у кого нет денег на счёте и есть кредитка и тех у кого есть деньги на счёте но нет кредитки. Тут важный момент состоит в том что данные могли быть взяты в момент когда клиент решил, например снять деньги для личных целей, но при этом вполне себе хорошо зарабатывает, посмтортим сколько из первой категории людей не имея денег на счёте и имея кредитки при этом зарабатывают ниже среднего значения по выборке

p.s. IsActiveMember перменая имеет странное описание на Kaggle - #Subjective, but for the concept


In [ ]:
print(f"Число клиентов с нулевым балансом, кредитками и зарплатой ниже среднего - {len(df.query('Balance == 0 & HasCrCard != 0 & EstimatedSalary < EstimatedSalary.mean()'))}")
display(df.query('Balance == 0 & HasCrCard != 0 & EstimatedSalary < EstimatedSalary.mean()').head())

Почему мы берем среднее в качестве порога отсечения для нашей новой фичи, причина проста - наша зарплата имеет равномерное распредение, а у него в асимптотике средние совпадает с медианой

In [ ]:
plt.title('EstimatedSalary', fontsize = 20)
plt.hist(df['EstimatedSalary'])
plt.xticks(rotation = 45)
plt.grid();

Преобразуем наши данные применив StandardScaler для последующей кластеризации с помощью K-means, попутно развлечения ради посмотрим на аппрокисмацию на двумерное пространство нашего датафрейма с помощью TSNE

In [ ]:
cols = df.columns
df_sc = df.copy().values 
scaler = sklearn.preprocessing.StandardScaler()

df_sc = scaler.fit_transform(df_sc)
df_sc = pd.DataFrame(df_sc)
df_sc.columns = cols
df_sc

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
digits_2d_tsne = tsne.fit_transform(df_sc.sample(len(df_sc), random_state = 0))
plt.figure(figsize=(10, 8))
plt.title('Двумерное представление нормированных данных', fontsize = 20)
plt.scatter(digits_2d_tsne[:, 0], digits_2d_tsne[:, 1], c = df_sc['target'].sample(len(df_sc), random_state = 0))
plt.colorbar()
plt.show()

Кластеризовать будем опираясь на метод "локтя" по метрике убывания суммы расстояний объектов кластеров от их центра

In [ ]:
from sklearn.cluster import KMeans

distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
X = df_sc 
for k in K: 
    kmeanModel = KMeans(n_clusters=k, random_state = 1).fit(X) 
    kmeanModel.fit(X)     
      
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / X.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / X.shape[0] 
    mapping2[k] = kmeanModel.inertia_
    
for key,val in mapping1.items(): 
    print(str(key)+' : '+str(val))
    
plt.plot(K, distortions, 'bx-') 
plt.xlabel('Количество кластеров K') 
plt.ylabel('Distortion') 
plt.title('Метод локтя с помощью Distortion') 
plt.show()

In [ ]:
for key,val in mapping2.items(): 
    print(str(key)+' : '+str(val))
    
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Количество кластеров K') 
plt.ylabel('Distortion') 
plt.title('Метод локтя с помощью Interia') 
plt.show()

Как мы видим оба метода для "локтевой прикидки" (Interia и Distortion) дают примерно одинаковый ответ - 3 кластера наиболее опитимальное разбиение. 

In [ ]:
kmeanModel = KMeans(n_clusters=3, random_state=0).fit(X) 
df['Cluster'] = kmeanModel.fit_predict(X)
df['Cluster'].value_counts()

Отлично, у нас примерно поровну разделились наблюдения, серьёзного перекоса в пользу одного из кластеров не случилось (как часто бывает, например с DBSCAN'ом)

Получим таким образом почти готовый для обучения модели датафрейм, добавив в OHE кодировке с префиксом 'Cluster' наши переменные, полученные после кластеризации

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Cluster'], prefix = 'Cluster')], axis = 1)
df.drop(columns = ['Cluster'], inplace = True)

Создадим новую перменную в которой будут клиенты с нулевым балансом, наличием кредитных карт и зарплатой ниже среднего по выборке (название новой переменной - four eyes priniciple aka FEP)

In [ ]:
df['FEP'] = np.where( ( (df['Balance'] == 0) & \
                       (df['EstimatedSalary'] < df['EstimatedSalary'].mean()) & \
                        df['HasCrCard'] > 0), 1, 0 )

Разобъём выборку в пропорции 80/20

In [ ]:
X = df[[i for i in df.columns if i != 'target']]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)

Простая функция для записи roc_auc в датафрейм

In [ ]:
def roc_auc_df (model, y_test, y_pred):
    global res
    res = res.append(pd.DataFrame([np.round(roc_auc_score(y_test, y_pred), 2)], columns = ['ROC_AUC'], index = [model]))
    return res

Переобъявим df_sc с новыми регрессорами чтобы обучить логистическую регрессию, для этого нужно будет удалить пару колонок из-за дамми-ловушки (я брал первые из категорий по порядку 'France' и 'Cluster_0')

In [ ]:
cols = df.columns
df_sc = df.copy().values 
scaler = sklearn.preprocessing.StandardScaler()

df_sc = scaler.fit_transform(df_sc)
df_sc = pd.DataFrame(df_sc)
df_sc.columns = cols
df_sc

Обучим логистическую регрессию на нормированных признаках

In [ ]:
X_sc = df_sc.drop(['target', 'France', 'Cluster_0'], axis=1) 
y_sc = df_sc['target']

X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(X_sc, y, test_size=0.2, random_state=1)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_sc, y_train_sc)
y_pred = lr.predict(X_test_sc)

print(classification_report(y_test_sc, y_pred))
print(f'LR_sc auc is {np.round(roc_auc_score(y_test, y_pred), 2)}')

res = pd.DataFrame([np.round(roc_auc_score(y_test, y_pred), 2)], columns = ['ROC_AUC'], index = ['LR'])
res

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(lr, X_test_sc, y_test, ax=ax1)
plot_roc_curve(lr, X_test_sc, y_test, ax=ax2);

Как мы видим accuracy выше чем соотношение классов 0/1 во всей выборке, то есть наша модель уже точнее (хоть и не сильно) простого констаного прогноза, но F1 для меньшего класса предсказуемо мала, будем ориентироватся не только на roc_auc но и на F1 для класса 1, так как нам бы хотелось иметь наиболее корректную модель, не "испорченную" дисбалансом классов

Опробуем случайный лес с параметром class_weight

In [ ]:
RF = RandomForestClassifier(random_state=1, class_weight='balanced')
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'RF roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}') 
f1('RF',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(RF, X_test, y_test, ax=ax1)
plot_roc_curve(RF, X_test, y_test, ax=ax2);

In [ ]:
XGB = XGBClassifier(random_state = 1)
XGB.fit(X_train, y_train)
y_pred = XGB.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'XGB roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('XGB',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(XGB, X_test, y_test, ax=ax1)
plot_roc_curve(XGB, X_test, y_test, ax=ax2);

In [ ]:
LGBM = LGBMClassifier(class_weight='balanced', random_state = 1)
LGBM.fit(X_train, y_train)
y_pred = LGBM.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'LGBM roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('LGBM',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(LGBM, X_test, y_test, ax=ax1)
plot_roc_curve(LGBM, X_test, y_test, ax=ax2);

In [ ]:
reg_cb = CatBoostClassifier(loss_function='Logloss', random_state = 1)

reg_cb.fit(X_train, y_train, verbose = False)

y_pred = reg_cb.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'CB roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('CB',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(reg_cb, X_test, y_test, ax=ax1)
plot_roc_curve(reg_cb, X_test, y_test, ax=ax2);

Итак, мы опробовали 4 алгоритма без настройки, один из них линейный (LR), мы можем перебрать все алгоритмы классификации, но в целях экономии времени, остановимся на этих четырёх

Попробуем процедуру under sampling'а с помощью TomekLinks и RandomOverSampling'a для всех вышеуказанных алгоритмов

In [ ]:
from imblearn.under_sampling import TomekLinks

TL = TomekLinks()
X_train_tl, y_train_tl = TL.fit_resample(X_train, y_train)

oversample = RandomOverSampler(sampling_strategy=1.0)
X_over, y_over = oversample.fit_resample(X_train, y_train)

RandomForest c undersampling

In [ ]:
reg = RandomForestClassifier(random_state=1, class_weight='balanced')
reg.fit(X_train_tl, y_train_tl)
y_pred = reg.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'RF TL roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('RF TL',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(reg, X_test, y_test, ax=ax1)
plot_roc_curve(reg, X_test, y_test, ax=ax2);

RadnomForest c RandomOS

In [ ]:
reg = RandomForestClassifier(random_state=1, class_weight='balanced')
reg.fit(X_over, y_over)
y_pred = reg.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'RF OS roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('RF OS',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(reg, X_test, y_test, ax=ax1)
plot_roc_curve(reg, X_test, y_test, ax=ax2);

XGB c undersampling

In [ ]:
XGB_tl = XGBClassifier(random_state = 1)
XGB_tl.fit(X_train_tl, y_train_tl)
y_pred = XGB_tl.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'XGB TL roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('XGB TL',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(XGB_tl, X_test, y_test, ax=ax1)
plot_roc_curve(XGB_tl, X_test, y_test, ax=ax2);

XGB c RandomOS

In [ ]:
XGB = XGBClassifier(random_state = 1)
XGB.fit(X_over, y_over)
y_pred = XGB.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'XGB OS roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('XGB OS',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(XGB, X_test, y_test, ax=ax1)
plot_roc_curve(XGB, X_test, y_test, ax=ax2);

LGBM c undersampling

In [ ]:
LGBM = LGBMClassifier(class_weight='balanced', random_state = 1)
LGBM.fit(X_train_tl, y_train_tl)
y_pred = LGBM.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'LGBM TL roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('LGBM TL',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(LGBM, X_test, y_test, ax=ax1)
plot_roc_curve(LGBM, X_test, y_test, ax=ax2);

LGBM с RandomOS

In [ ]:
LGBM_OS = LGBMClassifier(class_weight='balanced', random_state = 1)
LGBM_OS.fit(X_over, y_over)
y_pred = LGBM_OS.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'LGBM OS roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('LGBM OS',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(LGBM, X_test, y_test, ax=ax1)
plot_roc_curve(LGBM, X_test, y_test, ax=ax2);

Cat_Boost c undersampling

In [ ]:
reg_cb = CatBoostClassifier(loss_function='Logloss', random_state = 1)

reg_cb.fit(X_train_tl, y_train_tl, verbose = False)

y_pred = reg_cb.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'CB_TL roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('CB_TL',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(reg_cb, X_test, y_test, ax=ax1)
plot_roc_curve(reg_cb, X_test, y_test, ax=ax2);

CatBoost c RandomOS

In [ ]:
reg_cb = CatBoostClassifier(loss_function='Logloss', random_state = 1)

reg_cb.fit(X_over, y_over, verbose = False)

y_pred = reg_cb.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'CB_OS roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
f1('CB_OS',y_test,y_pred)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(reg_cb, X_test, y_test, ax=ax1)
plot_roc_curve(reg_cb, X_test, y_test, ax=ax2);

Так как мы имеем дело с дисбалансом классов, мы хотим иметь в каждом фолде на кросс-валидации нужную нам пропорцию классов для этого используем стратегию кросс-валидации StratifiedShuffleSplit на 5 фолдах

In [ ]:
# %%time
# lgbm = LGBMClassifier(class_weight='balanced', random_state = 1)

# skf = StratifiedShuffleSplit(n_splits = 5, random_state = 1)

# params = {'learning_rate': np.linspace(0.05, 0.5, 10),
#          'num_leaves':range(10, 100),
#          'n_estimators' : range(250, 500, 50),
#          'reg_alpha' : np.linspace(0, 10, 10),
#          'reg_lambda': np.linspace(0, 10, 10)}


# clf_lgbm = GridSearchCV(lgbm, params, scoring = 'roc_auc', cv = skf, verbose = True, n_jobs = -1)

# clf_lgbm.fit(X_over, y_over)

# best_params = clf_lgbm.best_estimator_.get_params()
# print('Best score: ', clf_lgbm.best_score_)
# print('Best params: ', best_params)

# print(classification_report(y_test, y_pred))
# print(f'LGBM_GS roc_auc_score is {np.round(roc_auc_score(y_test, y_pred), 2)}')
# f1('LGBM_GS',y_test,y_pred)

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# plot_precision_recall_curve(clf_lgbm, X_test, y_test, ax=ax1)
# plot_roc_curve(clf_lgbm, X_test, y_test, ax=ax2);

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 10,
            'categorical_feature': 'auto'}

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
skf = StratifiedShuffleSplit(n_splits = 5, random_state = 1)

clf = lgb.LGBMClassifier(max_depth=-1, random_state=1, silent=True, metric='None', n_jobs=-1, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=300,
    scoring='roc_auc',
    cv=skf,
    refit=True,
    random_state=1,
    verbose=True)

In [ ]:
gs.fit(X_over, y_over, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [ ]:
roc_auc_score(y_test, gs.predict(X_test))

##### grid search работает очень долго, а RandomizedSearch с нашими параметрами не дал более высокого качества

Наш лучший результат по метрике roc_auc обусловлен использованием LGBM при оверсэмплинге, мы можем посомтреть какие из регрессоров были наиболее "важными" для нашей модели с помощью разных методов.

1) Shap - теоретико-игровой подход под капотом

In [ ]:
import shap
explainer = shap.TreeExplainer(LGBM_OS)
shap_vals = explainer.shap_values(X_test, y_test)

shap.summary_plot(shap_values=shap_vals,
                features=X_test, plot_type='bar')

2) Обычный permutation_importance тест из sklearn

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(LGBM_OS, X_test, y_test, n_repeats=5, random_state=1, scoring='roc_auc')
pd.DataFrame({'Permutation_importance':result.importances_mean,\
              'Feature_name':X.columns}).\
            sort_values(by="Permutation_importance", ascending=False).head(15)

3) Стандартный метод оценки важности регрессоров, встроенный в LGBM (по сути, по оси x - количество сплитов в деревьях по каждой из фичей)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(LGBM_OS.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)', fontsize = 20)
plt.show()

Лучший сабмит что я видел по accuracy и F1 для классов 0 и 1 - https://www.kaggle.com/affanamin/handling-imbalanced-data-in-deeplearning, задача была побить его не используя DL (хотя там TF)

По поводу предобработки фичей, к сожалению кластеризация нам не пригодилась
##### то же самое по поводу кластеризации прямо подтверждает XGBOOST 


In [ ]:
_ = xgb.plot_importance(XGB, height=0.7, grid=True, max_num_features = None)

In [ ]:
Вводы